#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [6]:
import pandas as pd
import pymongo

In [7]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
database = client['exercices']
collection = database['kickstarter']

In [8]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [9]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [32]:
print(len(df_ks))
df_ks = df_ks.drop_duplicates(subset=["name"])
len(df_ks)

150000


149478

### Importer les données

In [12]:
import numpy as np

def createDocsFromDF(df, collection=None, insertToDB=False):
    docs = [] 
    fields = [col for col in df.columns]
    for i in range(len(df)):
        doc = {col:df[col][i] for col in df.columns if col != 'index'}
        for key, val in doc.items():
            # we have to do this, because mongo does not recognize these np. types
            if type(val) == np.int64:
                doc[key] = int(val)
            if type(val) == np.float64:
                doc[key] = float(val)
            if type(val) == np.bool_:
                doc[key] = bool(val)
        docs.append(doc) 
    if insertToDB and collection:
        database.collection.insert_many(docs)
    return docs

createDocsFromDF(df_ks, collection="kickstarter", insertToDB=True)

[{'ID': 872782264,
  'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
  'category': 'Rock',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2011-09-16',
  'goal': 2000.0,
  'launched': '2011-08-17 06:31:31',
  'pledged': 1145.0,
  'state': 'canceled',
  'backers': 24,
  'country': 'US',
  'usd pledged': 1145.0,
  'usd_pledged_real': 1145.0,
  '_id': ObjectId('605c6ac02b63b6bd183cf18f')},
 {'ID': 1326492673,
  'name': 'Ohceola jewelry',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'currency': 'USD',
  'deadline': '2012-08-22',
  'goal': 18000.0,
  'launched': '2012-07-23 20:46:48',
  'pledged': 1851.0,
  'state': 'failed',
  'backers': 28,
  'country': 'US',
  'usd pledged': 1851.0,
  'usd_pledged_real': 1851.0,
  '_id': ObjectId('605c6ac02b63b6bd183cf190')},
 {'ID': 1688410639,
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '20

## Question 1  

In [20]:
res = database.collection.find().sort("pledged", -1).limit(5)
for i in res:
    print(f'{i["name"]} : {i["pledged"]}')

True North ï¿½ Repurposed Sailcloth Products : 9996.0
Scott Davis - Solo Record : 9995.0
Polco: A Digital Platform for Better Civic Communication : 9994.0
2016 Gaming Calendar : 9994.0
The Girl in The Mountain - Film : 999.0


## Question 2

In [29]:
res = database.collection.find({"$where":"this.goal <= this.pledged"}).count()
print(i)

D:\Users\redad\miniconda3\envs\artificial_env\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


{'_id': ObjectId('605c6ac02b63b6bd183cf29a'), 'ID': 377490181, 'name': 'Wandlung - 100% Funded with new Stretch Goals!', 'category': "Children's Books", 'main_category': 'Publishing', 'currency': 'USD', 'deadline': '2013-12-28', 'goal': 2000.0, 'launched': '2013-12-02 22:00:11', 'pledged': 2365.0, 'state': 'successful', 'backers': 35, 'country': 'US', 'usd pledged': 2365.0, 'usd_pledged_real': 2365.0}


## Question 3

## Question 4

## Question 5

## Question 6 